# Cs 2316 Final Project - Data retrieval and cleaning

In [ ]:
import requests
from pprint import pprint
import yfinance as yf
import pandas as pd
import time
import numpy as np
import regex

In [2]:
def sectordata():
    import yfinance as yf
    from pprint import pprint
    import pandas as pd

    #using the yahoo finance package to get data for the dow jones industrial adverage and various sector index funds
    
    data = yf.download('^DJI','2020-01-22','2021-10-11')   #retrieves and stores dow jones open values
    data = data.iloc[:,0]
    data.to_csv("DOW_price2.csv")
    sector_data = pd.DataFrame(data,columns = ["Open"])
    
    i=0    #loop retrieves data for the selected index funds; i used to initialize pandas df for other values to be added to
    lisq = [("XLE","Energy"),("XLF","Financial Services"),("XLK","Technology"),("XLRE","Real Estate"),("XLC","Communication Services"),("XLU","Utilities"),("XLP","Consumer Staples"),("XLV","Healthcare"),("XLB","Basic Materials")]
    for tick, name in lisq:
        i += 1
        data = yf.download('{}'.format(tick),'2020-01-22','2021-10-11')
        data = data.iloc[:,0]
        if i == 1:
            sector_data["Energy"] = pd.DataFrame(data)
        else:
            sector_data["{}".format(name)] = data
    sector_data.to_csv("Sector_Prices2.csv")
    pprint(sector_data)
sectordata()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                    Open     Energy  Financial Services  Technology  \
Date                                                                  
2020-01-22  29263.630859  57.950001           30.870001   97.690002   
2020-01-23  29111.019531  56.900002           30.700001   97.529999   
2020-01-24  29230.390625  

I used the Yahoo finance package in order to download market data for the Dow Jones Industrial average as well as index funds for the largest sectors in the economy. I choose to use the Open price as the data points for this analysis.

In [6]:
import requests
import pandas as pd
import datetime

#using the guardian's api to find articles mentioning covid or coronavirus

#finding number of pages retuned from the api to use in loop later
res = requests.get("https://content.guardianapis.com/search?q=covid%20OR%20caronavirus&api-key=ad1362b7-3b61-4cc4-8fde-f1c0e57c3cd9")
data = res.json()
pages = data["response"]["pages"]


used = [] #initializing
dj = 0
outdic = {}
s = requests.Session()  #creating a session for the many get requests in the loop
start = datetime.datetime.now()  #used to get runtime; testing improvement from adding a requests session
for i in range(pages):  #querying each page of the api and creating a dictionary with dates as keys and the # of articles as values
    gurls = "https://content.guardianapis.com/search?page={}&q=covid%20OR%20caronavirus&api-key=ad1362b7-3b61-4cc4-8fde-f1c0e57c3cd9".format(i+1)
    resp = s.get(gurls)
    resp = resp.json()
    pagedata = resp["response"]["results"]
    for each in pagedata:
        datex = each["webPublicationDate"]
        date = datex[:10]
        if date in used:
            outdic[date] += 1
        else:
            used += [date]
            outdic[date] = 1

pd.DataFrame.from_dict(data=outdic, orient='index').to_csv("api_dic_sessiontest.csv", header=False)
finish = datetime.datetime.now() - start
print("Runtime:")
print(finish)
print(outdic)

Runtime:
0:09:48.609597
{'2022-02-21': 33, '2022-02-06': 12, '2022-03-08': 6, '2022-02-19': 16, '2022-01-17': 28, '2022-01-26': 23, '2022-02-20': 18, '2022-03-05': 4, '2022-03-13': 11, '2021-12-30': 34, '2022-02-08': 16, '2022-03-03': 8, '2022-03-02': 13, '2022-01-11': 47, '2022-02-16': 21, '2021-12-03': 39, '2022-03-12': 8, '2022-01-09': 19, '2021-12-08': 43, '2022-02-27': 8, '2022-02-14': 15, '2022-02-23': 23, '2022-03-10': 10, '2022-01-20': 29, '2022-01-04': 48, '2022-03-11': 12, '2022-01-30': 16, '2022-01-16': 18, '2022-02-11': 21, '2022-02-10': 22, '2022-02-09': 32, '2022-01-27': 35, '2022-01-21': 33, '2022-02-15': 20, '2022-02-03': 22, '2021-12-27': 23, '2022-01-19': 31, '2022-01-07': 42, '2022-02-26': 9, '2022-01-06': 34, '2022-03-14': 24, '2022-02-12': 8, '2022-02-25': 22, '2022-01-05': 42, '2022-03-07': 6, '2022-03-06': 7, '2022-02-22': 17, '2022-01-25': 42, '2022-02-02': 23, '2022-02-01': 20, '2022-01-12': 52, '2021-12-28': 27, '2022-01-10': 31, '2022-03-04': 8, '2021-11-21':

Data from this API was already cleaned, but I had to query the api to find articles mentioning covid or corona virus and retrieve the dates of these articles. I initially used individual get requests to retrieve data but got timeout errors before searching all the pages from the api so I created a session in order to maintain the same connection for many get requests.

In [1]:
import pandas as pd
with open("covid_data.csv") as fcov:
    dd = pd.read_csv(fcov,delimiter=',')
    ddd = dd.fillna(0)
    ddd = ddd.iloc[:, :12]   #getting necessary values
    colnames = ddd.columns
    colnames = colnames[2:]
    for name in colnames:
        for i in ddd.index:
            if ddd.loc[i, name] < 0: #replacign negative numbers with zero as negative cases/deaths doesn't make sesnse
                ddd.loc[i, name] = 0

    ddd["submission_date"] = pd.to_datetime(ddd["submission_date"])
    data = ddd.sort_values(by = "submission_date")  #combining data from different states on the same date into a national total
    data = data.groupby(["submission_date"]).sum()
    
    data.to_csv("COV_dat2.csv")
    print(data.head())

                 tot_cases  conf_cases  prob_cases  new_case  pnew_case  \
submission_date                                                           
2020-01-22               7         6.0         1.0         7        1.0   
2020-01-23               8         6.0         1.0         1        0.0   
2020-01-24               9         6.0         1.0         1        0.0   
2020-01-25               9         6.0         1.0         0        0.0   
2020-01-26              12         6.0         1.0         3        0.0   

                 tot_death  conf_death  prob_death  new_death  pnew_death  
submission_date                                                            
2020-01-22               0         0.0         0.0          0         0.0  
2020-01-23               0         0.0         0.0          0         0.0  
2020-01-24               0         0.0         0.0          0         0.0  
2020-01-25               0         0.0         0.0          0         0.0  
2020-01-26        

I used a csv file which contained covid data for all states organized by date. I have filled any Nans, corrected all negative numbers by making them 0, and selected the columns of data I want from this dataset. I also compiled the data by date to eliminate the seperation by state as this isn't needed in my analysis.